In [1]:
stock = 'VZ'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    50.065224
2018-07-23    50.193798
2018-07-24    50.945469
2018-07-25    50.352047
2018-07-26    50.619087
2018-07-27    51.439987
2018-07-30    51.914730
2018-07-31    51.074043
2018-08-01    51.163059
2018-08-02    51.410316
2018-08-03    51.697140
2018-08-06    51.904839
2018-08-07    51.885056
2018-08-08    52.122429
2018-08-09    52.429028
2018-08-10    51.894951
2018-08-13    51.647690
2018-08-14    52.290562
2018-08-15    52.656509
2018-08-16    53.675220
2018-08-17    54.189522
2018-08-20    54.051060
2018-08-21    54.308208
2018-08-22    53.546646
2018-08-23    53.685112
2018-08-24    54.179630
2018-08-27    54.248863
2018-08-28    54.051060
2018-08-29    53.971931
2018-08-30    54.149960
                ...    
2018-09-17    54.100510
2018-09-18    53.793907
2018-09-19    52.913658
2018-09-20    53.358730
2018-09-21    53.823574
2018-09-24    52.953224
2018-09-25    52.468590
2018-09-26    52.557606
2018-09-27    53.002674
2018-09-28    52.804867
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


54.03222476230824
[54.46725494 54.16052098 54.79130701 53.64567437 53.0963665 ]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")